# 1. Install required libraries and packages

In [473]:
pip install osmnx geopandas pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1.1 install packages

In [474]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import requests
import time
import logging
from sqlalchemy import create_engine, text
import psycopg2
import warnings

warnings.filterwarnings("ignore")

# Display all rows and columns in pandas DataFrames
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)


In [475]:
# disable disk caching
ox.settings.use_cache = False

# 2. get hospital/clinic data from OSM

In [476]:
tags = {
    "healthcare": ["hospital","clinic"],  # amenity tagging
    "amenity": ["hospital","clinic"]  # healthcare tagging
 }
hospital_gdf = ox.features_from_place("Berlin, Germany", tags)
hospital_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 251 entries, ('node', np.int64(669088712)) to ('way', np.int64(1433767664))
Columns: 133 entries, geometry to fence_type
dtypes: geometry(1), object(132)
memory usage: 271.9+ KB


In [477]:
hospital_gdf.head(10)

geometry addr:city addr:country  \
element id                                                             
node    669088712   POINT (13.50017 52.43969)    Berlin           DE   
        694302689   POINT (13.49094 52.51514)    Berlin          NaN   
        872223518   POINT (13.41986 52.49885)       NaN          NaN   
        874357625    POINT (13.4133 52.50279)    Berlin           DE   
        911925361   POINT (13.41698 52.57214)    Berlin          NaN   
        933513566    POINT (13.2153 52.43929)    Berlin           DE   
        1038248666  POINT (13.46807 52.50449)    Berlin           DE   
        1168212013  POINT (13.43526 52.42279)    Berlin           DE   
        1217659471  POINT (13.23851 52.44415)    Berlin           DE   
        1272472319  POINT (13.45114 52.51516)    Berlin           DE   

                   addr:housenumber addr:postcode          addr:street  \
element id                                                               
node    669088712               152         12487            Sterndamm   
        694302689                 4         10365         Schottstraße   
        872223518               NaN           NaN                  NaN   
        874357625               158         10969        Oranienstraße   
        911925361                 7         13187        Damerowstraße   
        933513566               1-3         14129    Breisgauer Straße   
        1038248666                1         10245        Sonntagstraße   
        1168212013             9-11         12349           Alt-Buckow   
        1217659471              111         14163  Fischerhüttenstraße   
        1272472319               23         10243       Kadiner Straße   

                       addr:suburb amenity  check_date healthcare  \
element id                                                          
node    669088712     Johannisthal  clinic  2023-08-13     clinic   
        694302689              NaN  clinic         NaN     clinic   
        872223518              NaN  clinic         NaN     clinic   
        874357625        Kreuzberg  clinic         NaN     clinic   
        911925361           Pankow  clinic         NaN     clinic   
        933513566    Schlachtensee  clinic         NaN     clinic   
        1038248666  Friedrichshain  clinic  2024-11-28     clinic   
        1168212013          Buckow  clinic         NaN     clinic   
        1217659471      Zehlendorf  clinic         NaN   dialysis   
        1272472319  Friedrichshain  clinic         NaN     clinic   

                                                     name wheelchair  \
element id                                                             
node    669088712                  Ärztehaus Johannisthal        yes   
        694302689             Ärztehaus am Roedeliusplatz    limited   
        872223518                               Ärztehaus        yes   
        874357625                      MVZ am Moritzplatz        yes   
        911925361                 Ärztehaus Damerowstraße    limited   
        933513566              Ärztehaus am Schlachtensee        yes   
        1038248666     Chirurgie & Orthopädie am Ostkreuz        yes   
        1168212013                            MVZ-Vitalis        yes   
        1217659471               Nierenzentrum Zehlendorf        NaN   
        1272472319  DTZ Berlin - Ärztehaus Kadiner Straße        yes   

                                                          opening_hours  \
element id                                                                
node    669088712                                                   NaN   
        694302689                                                   NaN   
        872223518   Mo,Tu,Th 08:30-13:00,14:00-18:00; We,Fr 08:30-14:00   
        874357625               Mo,Tu,Th 08:00-18:00; We,Fr 08:00-12:00   
        911925361                                                   NaN   
        933513566                                               

In [478]:
# Ensure geometry type is Point for lat/lon extraction

hospital_gdf = hospital_gdf.to_crs(epsg=4326)

## 2.1 extract longitude and latitude

In [479]:
hospital_gdf['geometry'] = hospital_gdf['geometry'].apply(lambda geom: geom if geom.geom_type == 'Point' else geom.representative_point())
#Extract latitude and longitude
hospital_gdf["latitude"] = hospital_gdf.geometry.y
hospital_gdf["longitude"] = hospital_gdf.geometry.x
hospital_gdf.head(3)

geometry addr:city addr:country  \
element id                                                            
node    669088712  POINT (13.50017 52.43969)    Berlin           DE   
        694302689  POINT (13.49094 52.51514)    Berlin          NaN   
        872223518  POINT (13.41986 52.49885)       NaN          NaN   

                  addr:housenumber addr:postcode   addr:street   addr:suburb  \
element id                                                                     
node    669088712              152         12487     Sterndamm  Johannisthal   
        694302689                4         10365  Schottstraße           NaN   
        872223518              NaN           NaN           NaN           NaN   

                  amenity  check_date healthcare                         name  \
element id                                                                      
node    669088712  clinic  2023-08-13     clinic       Ärztehaus Johannisthal   
        694302689  clinic         NaN     clinic  Ärztehaus am Roedeliusplatz   
        872223518  clinic         NaN     clinic                    Ärztehaus   

                  wheelchair  \
element id                     
node    669088712        yes   
        694302689    limited   
        872223518        yes   

                                                         opening_hours  \
element id                                                               
node    669088712                                                  NaN   
        694302689                                                  NaN   
        872223518  Mo,Tu,Th 08:30-13:00,14:00-18:00; We,Fr 08:30-14:00   

                  healthcare:speciality phone website wheelchair:description  \
element id                                                                     
node    669088712                   NaN   NaN     NaN                    NaN   
        694302689                   NaN   NaN     NaN                    NaN   
        872223518                   NaN   NaN     NaN                    NaN   

                  check_date:opening_hours contact:phone contact:website  \
element id                                                                 
node    669088712                      NaN           NaN             NaN   
        694302689                      NaN           NaN             NaN   
        872223518                      NaN           NaN             NaN   

                  email operator operator:type contact:email  fax source  \
element id                                                                 
node    669088712   NaN      NaN           NaN           NaN  NaN    NaN   
        694302689   NaN      NaN           NaN           NaN  NaN    NaN   
        872223518   NaN      NaN           NaN           NaN  NaN    NaN   

                  toilets:wheelchair health_facility:type  \
element id                                                  
node    669088712                NaN                  NaN   
        694302689                NaN                  NaN   
        872223518                NaN                  NaN   

                  medical_system:western opening_hours:signed description  \
element id                                                                  
node    669088712                    NaN                  NaN         NaN   
        694302689                    NaN                  NaN         NaN   
        872223518                    NaN                  NaN         NaN   

                  level note emergency opening_hours:dr_weinhold access  \
element id                                                                
node    669088712   NaN  NaN       NaN                       NaN    NaN   
        694302689   NaN  NaN       NaN                       NaN    NaN   
        872223518   NaN  NaN       NaN                       NaN    NaN   

                  alt_name start_date beds  \
element id                                   
node    669088712 

## 2.2 set Country to Germany

In [480]:

# Update only where city is Berlin AND country is null/NaN/empty
hospital_gdf.loc[(hospital_gdf["addr:city"] == "Berlin") & (hospital_gdf["addr:country"].isna()), "addr:country"] = "DE"
hospital_gdf.head(20)

geometry addr:city addr:country  \
element id                                                             
node    669088712   POINT (13.50017 52.43969)    Berlin           DE   
        694302689   POINT (13.49094 52.51514)    Berlin           DE   
        872223518   POINT (13.41986 52.49885)       NaN          NaN   
        874357625    POINT (13.4133 52.50279)    Berlin           DE   
        911925361   POINT (13.41698 52.57214)    Berlin           DE   
        933513566    POINT (13.2153 52.43929)    Berlin           DE   
        1038248666  POINT (13.46807 52.50449)    Berlin           DE   
        1168212013  POINT (13.43526 52.42279)    Berlin           DE   
        1217659471  POINT (13.23851 52.44415)    Berlin           DE   
        1272472319  POINT (13.45114 52.51516)    Berlin           DE   
        1336685638  POINT (13.20446 52.53695)       NaN          NaN   
        1618861746  POINT (13.57587 52.44728)       NaN          NaN   
        1724523076  POINT (13.29126 52.44445)    Berlin           DE   
        1732901969  POINT (13.35817 52.54662)    Berlin           DE   
        1734428743  POINT (13.40265 52.58092)    Berlin           DE   
        1974936511  POINT (13.29298 52.49106)       NaN          NaN   
        2021436781  POINT (13.54199 52.52751)    Berlin           DE   
        2105895889  POINT (13.38609 52.45556)    Berlin           DE   
        2126141390  POINT (13.28372 52.59139)    Berlin           DE   
        2335086445  POINT (13.45513 52.51221)    Berlin           DE   

                   addr:housenumber addr:postcode           addr:street  \
element id                                                                
node    669088712               152         12487             Sterndamm   
        694302689                 4         10365          Schottstraße   
        872223518               NaN           NaN                   NaN   
        874357625               158         10969         Oranienstraße   
        911925361                 7         13187         Damerowstraße   
        933513566               1-3         14129     Breisgauer Straße   
        1038248666                1         10245         Sonntagstraße   
        1168212013             9-11         12349            Alt-Buckow   
        1217659471              111         14163   Fischerhüttenstraße   
        1272472319               23         10243        Kadiner Straße   
        1336685638              NaN           NaN                   NaN   
        1618861746              NaN           NaN                   NaN   
        1724523076               93         12205      Unter den Eichen   
        1732901969              151         13353          Müllerstraße   
        1734428743                4         13156  Hermann-Hesse-Straße   
        1974936511              NaN           NaN                   NaN   
        2021436781               10         12681   Helene-Weigel-Platz   
        2105895889              227         12099      Tempelhofer Damm   
        2126141390                5         13507          Schloßstraße   
        2335086445            43-45         10245     Grünberger Straße   

                       addr:suburb amenity  check_date healthcare  \
element id                                                          
node    669088712     Johannisthal  clinic  2023-08-13     clinic   
        694302689              NaN  clinic         NaN     clinic   
        872223518              NaN  clinic         NaN     clinic   
        874357625        Kreuzberg  clinic         NaN     clinic   
        911925361           Pankow  clinic         NaN     clinic   
        933513566    Schlachtensee  clinic         NaN     clinic   
        1038248666  Friedrichshain  clinic  2024-11-28     clinic   
        1168212013          Buckow  clinic         NaN     clinic   
        1217659471      Zehlendorf  clinic         NaN   dialysis   
        1272472319  Friedrichshain  clinic

# 3. select required columns

In [481]:
# Select the columns  and rename accordingly

selected_columns = [
         "name", "operator", "brand", "addr:country", "addr:city", "addr:street", "addr:housenumber", "addr:postcode", "addr:suburb", 
         "phone","email", "website", "wheelchair", "toilets:wheelchair", "beds", "emergency", "healthcare:speciality",
         "opening_hours","latitude", "longitude","geometry", "source","amenity", "healthcare"]
      

In [482]:
hospital_df = hospital_gdf[selected_columns]
hospital_df.head(3)

name operator brand addr:country  \
element id                                                                   
node    669088712       Ärztehaus Johannisthal      NaN   NaN           DE   
        694302689  Ärztehaus am Roedeliusplatz      NaN   NaN           DE   
        872223518                    Ärztehaus      NaN   NaN          NaN   

                  addr:city   addr:street addr:housenumber addr:postcode  \
element id                                                                 
node    669088712    Berlin     Sterndamm              152         12487   
        694302689    Berlin  Schottstraße                4         10365   
        872223518       NaN           NaN              NaN           NaN   

                    addr:suburb phone email website wheelchair  \
element id                                                       
node    669088712  Johannisthal   NaN   NaN     NaN        yes   
        694302689           NaN   NaN   NaN     NaN    limited   
        872223518           NaN   NaN   NaN     NaN        yes   

                  toilets:wheelchair beds emergency healthcare:speciality  \
element id                                                                  
node    669088712                NaN  NaN       NaN                   NaN   
        694302689                NaN  NaN       NaN                   NaN   
        872223518                NaN  NaN       NaN                   NaN   

                                                         opening_hours  \
element id                                                               
node    669088712                                                  NaN   
        694302689                                                  NaN   
        872223518  Mo,Tu,Th 08:30-13:00,14:00-18:00; We,Fr 08:30-14:00   

                    latitude  longitude                   geometry source  \
element id                                                                  
node    669088712  52.439692  13.500167  POINT (13.50017 52.43969)    NaN   
        694302689  52.515142  13.490941  POINT (13.49094 52.51514)    NaN   
        872223518  52.498845  13.419860  POINT (13.41986 52.49885)    NaN   

                  amenity healthcare  
element id                            
node    669088712  clinic     clinic  
        694302689  clinic     clinic  
        872223518  clinic     clinic

# 4. sanitary checks

## 4.1 name missing

In [483]:
nan_names = hospital_df[hospital_df["name"].isna()]

print(nan_names)
print(len(nan_names), "rows with NaN name")

                    name operator brand addr:country addr:city  \
element id                                                       
node    12896993603  NaN      NaN   NaN          NaN       NaN   
way     98338145     NaN      NaN   NaN           DE    Berlin   
        318581439    NaN      NaN   NaN           DE    Berlin   
        318581473    NaN      NaN   NaN           DE    Berlin   
        318581474    NaN      NaN   NaN           DE    Berlin   
        970838194    NaN      NaN   NaN          NaN       NaN   

                          addr:street addr:housenumber addr:postcode  \
element id                                                             
node    12896993603   Oberspreestraße              186         12557   
way     98338145     Karl-Marx-Straße               27         12043   
        318581439           Clayallee             225A         14195   
        318581473           Clayallee             225B         14195   
        318581474           Clayallee  

6 rows with missing hospital name

## 4.2 top 50 hospital names

In [484]:
print("\nTop 10 hospitals:")
print(hospital_df["name"].value_counts().head(10))


Top 10 hospitals:
name
Ärztehaus                           24
Gesundheitszentrum                   3
MVZ Pankow Innere Medizin            2
Ärztehaus Johannisthal               1
Polikum                              1
Ärztehaus Dr. Karl Kollwitz          1
Ambulatorium Schillingstraße 12      1
Helios Klinikum Emil von Behring     1
Krankenhaus Waldfriede               1
KfH-Nierenzentrum                    1
Name: count, dtype: int64


most of names only appear once, but Ärztehaus, Gesundheitszentrum and MVZ Pankow Innere Medizin have multiple entries

## 4.3 difference of healtcare and amenity

In [485]:
# Find rows where the two columns are different
 
diff = hospital_df[hospital_df["amenity"] != hospital_df["healthcare"]][["name", "amenity", "healthcare"]]


print(diff)

                                                                                                         name  \
element  id                                                                                                     
node     1217659471                                                                  Nierenzentrum Zehlendorf   
         2557712726                                                                         Magnet MVZ Berlin   
         2821025302                                                              Nierenzentrum Charlottenburg   
         2851155103                                                   Zahnklinik/Zahnärztlicher Notfalldienst   
         5181354049                                                                         Augen-Tagesklinik   
         5418942204                                                                 Nierenzentrum Lichtenberg   
         5849666900                                                                        Klini

In [486]:
# Print total count
print("\nTotal mismatches:", len(diff))


Total mismatches: 35


## 5. rename columns

In [487]:
rename_map = {
    "addr:street": "street",
    "addr:housenumber": "housenumber",
    "addr:postcode": "postcode",
    "addr:city": "city",
    "addr:country": "country",
    "addr:suburb": "neighborhood",
    "healthcare:speciality": "speciality",
    "toilets:wheelchair": "toilets_wheelchair",
    "amenity": "amenity_tag",
    "healthcare": "healthcare_tag"}

In [488]:
# Rename the columns
hospital_df = hospital_df.rename(columns=rename_map)
hospital_df.head(3)

name operator brand country    city  \
element id                                                                      
node    669088712       Ärztehaus Johannisthal      NaN   NaN      DE  Berlin   
        694302689  Ärztehaus am Roedeliusplatz      NaN   NaN      DE  Berlin   
        872223518                    Ärztehaus      NaN   NaN     NaN     NaN   

                         street housenumber postcode  neighborhood phone  \
element id                                                                 
node    669088712     Sterndamm         152    12487  Johannisthal   NaN   
        694302689  Schottstraße           4    10365           NaN   NaN   
        872223518           NaN         NaN      NaN           NaN   NaN   

                  email website wheelchair toilets_wheelchair beds emergency  \
element id                                                                     
node    669088712   NaN     NaN        yes                NaN  NaN       NaN   
        694302689   NaN     NaN    limited                NaN  NaN       NaN   
        872223518   NaN     NaN        yes                NaN  NaN       NaN   

                  speciality  \
element id                     
node    669088712        NaN   
        694302689        NaN   
        872223518        NaN   

                                                         opening_hours  \
element id                                                               
node    669088712                                                  NaN   
        694302689                                                  NaN   
        872223518  Mo,Tu,Th 08:30-13:00,14:00-18:00; We,Fr 08:30-14:00   

                    latitude  longitude                   geometry source  \
element id                                                                  
node    669088712  52.439692  13.500167  POINT (13.50017 52.43969)    NaN   
        694302689  52.515142  13.490941  POINT (13.49094 52.51514)    NaN   
        872223518  52.498845  13.419860  POINT (13.41986 52.49885)    NaN   

                  amenity_tag healthcare_tag  
element id                                    
node    669088712      clinic         clinic  
        694302689      clinic         clinic  
        872223518      clinic         clinic

In [489]:
hospital_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 251 entries, ('node', np.int64(669088712)) to ('way', np.int64(1433767664))
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   name                245 non-null    object  
 1   operator            85 non-null     object  
 2   brand               5 non-null      object  
 3   country             178 non-null    object  
 4   city                175 non-null    object  
 5   street              192 non-null    object  
 6   housenumber         189 non-null    object  
 7   postcode            181 non-null    object  
 8   neighborhood        123 non-null    object  
 9   phone               57 non-null     object  
 10  email               17 non-null     object  
 11  website             101 non-null    object  
 12  wheelchair          123 non-null    object  
 13  toilets_wheelchair  22 non-null     object  
 14  beds                3 non-nu

# 6. retrieve district and district_id and neighborhood for missings

In [490]:
from geopy.geocoders import Nominatim
from time import sleep

In [491]:


def fetch_location_info(df, lat_col="latitude", lon_col="longitude", level="district", user_agent="berlin-venues-scraper/1.0", delay=1):
    """
    Fetch district or neighborhood from Nominatim for a DataFrame with lat/lon columns.
    
    Parameters:
        df: pd.DataFrame with latitude and longitude columns
        lat_col, lon_col: names of the lat/lon columns
        level: "district" or "neighborhood"
        user_agent: User-Agent string for Nominatim
        delay: delay in seconds between requests
    
    Returns:
        pd.Series with district or neighborhood names
    """
    def get_info(lat, lon):
        url = "https://nominatim.openstreetmap.org/reverse"
        params = {"lat": lat, "lon": lon, "format": "json", "addressdetails": 1}
        headers = {"User-Agent": user_agent}
        try:
            r = requests.get(url, params=params, headers=headers, timeout=10)
            r.raise_for_status()
            data = r.json()
            address = data.get("address", {})

            if level == "district":
                # Only official Bezirke
                return (
                    address.get("city_district")
                    or address.get("borough")
                    or address.get("county")
                    or address.get("state_district")
                )
            elif level == "neighborhood":
                # Include suburb / neighbourhood / city_district
                return (
                    address.get("suburb")
                    or address.get("city_district")
                    or address.get("borough")
                    or address.get("neighbourhood")
                )
            else:
                return None
        except requests.exceptions.RequestException as e:
            logging.warning(f"Error fetching {level} for ({lat}, {lon}): {e}")
            return None

    # Apply with throttling
    results = []
    for i, row in df.iterrows():
        lat, lon = row[lat_col], row[lon_col]
        if pd.notna(lat) and pd.notna(lon):
            results.append(get_info(lat, lon))
            time.sleep(delay)
        else:
            results.append(None)
    return pd.Series(results, index=df.index)

# Usage examples:
hospital_df["district"] = fetch_location_info(hospital_df, level="district")
hospital_df["neighborhood_new"] = fetch_location_info(hospital_df, level="neighborhood")


In [492]:
hospital_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 251 entries, ('node', np.int64(669088712)) to ('way', np.int64(1433767664))
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   name                245 non-null    object  
 1   operator            85 non-null     object  
 2   brand               5 non-null      object  
 3   country             178 non-null    object  
 4   city                175 non-null    object  
 5   street              192 non-null    object  
 6   housenumber         189 non-null    object  
 7   postcode            181 non-null    object  
 8   neighborhood        123 non-null    object  
 9   phone               57 non-null     object  
 10  email               17 non-null     object  
 11  website             101 non-null    object  
 12  wheelchair          123 non-null    object  
 13  toilets_wheelchair  22 non-null     object  
 14  beds                3 non-nu

In [493]:
# District mapping (official codes as strings)
district_mapping = {
    'Mitte': '11001001',
    'Friedrichshain-Kreuzberg': '11002002',
    'Pankow': '11003003',
    'Charlottenburg-Wilmersdorf': '11004004',
    'Spandau': '11005005',
    'Steglitz-Zehlendorf': '11006006',
    'Tempelhof-Schöneberg': '11007007',
    'Neukölln': '11008008',
    'Treptow-Köpenick': '11009009',
    'Marzahn-Hellersdorf': '11010010',
    'Lichtenberg': '11011011',
    'Reinickendorf': '11012012'
}

# Apply mapping to create district_id column
hospital_df['district_id'] = (
    hospital_df['district']
    .map(district_mapping)
    .astype(str)
)
 

In [494]:
#for missing fill with retrieve information from neighborhood and drop the temporary column
hospital_df["neighborhood"] = hospital_df["neighborhood"].fillna(hospital_df["neighborhood_new"])
hospital_df.drop(columns=["neighborhood_new"], inplace=True)
hospital_df.head(10)

name  \
element id                                                  
node    669088712                  Ärztehaus Johannisthal   
        694302689             Ärztehaus am Roedeliusplatz   
        872223518                               Ärztehaus   
        874357625                      MVZ am Moritzplatz   
        911925361                 Ärztehaus Damerowstraße   
        933513566              Ärztehaus am Schlachtensee   
        1038248666     Chirurgie & Orthopädie am Ostkreuz   
        1168212013                            MVZ-Vitalis   
        1217659471               Nierenzentrum Zehlendorf   
        1272472319  DTZ Berlin - Ärztehaus Kadiner Straße   

                                                                           operator  \
element id                                                                            
node    669088712                                                               NaN   
        694302689                                                               NaN   
        872223518                                                               NaN   
        874357625                                                               NaN   
        911925361                                                               NaN   
        933513566                                                               NaN   
        1038248666  Dr. med. Christoph Graf von Matuschka  Dr. med. Christoph Lemke   
        1168212013                                                              NaN   
        1217659471                                                              NaN   
        1272472319                                                              NaN   

                   brand country    city               street housenumber  \
element id                                                                  
node    669088712    NaN      DE  Berlin            Sterndamm         152   
        694302689    NaN      DE  Berlin         Schottstraße           4   
        872223518    NaN     NaN     NaN                  NaN         NaN   
        874357625    NaN      DE  Berlin        Oranienstraße         158   
        911925361    NaN      DE  Berlin        Damerowstraße           7   
        933513566    NaN      DE  Berlin    Breisgauer Straße         1-3   
        1038248666   NaN      DE  Berlin        Sonntagstraße           1   
        1168212013   NaN      DE  Berlin           Alt-Buckow        9-11   
        1217659471   NaN      DE  Berlin  Fischerhüttenstraße         111   
        1272472319   NaN      DE  Berlin       Kadiner Straße          23   

                   postcode    neighborhood             phone  \
element id                                                      
node    669088712     12487    Johannisthal               NaN   
        694302689     10365     Lichtenberg               NaN   
        872223518       NaN       Kreuzberg               NaN   
        874357625     10969       Kreuzberg    +49 30 6146017   
        911925361     13187          Pankow               NaN   
        933513566     14129   Schlachtensee               NaN   
        1038248666    10245  Friedrichshain               NaN   
        1168212013    12349          Buckow   +49 30 67065690   
        1217659471    14163      Zehlendorf      +49308139121   
        1272472319    10243  Friedrichshain  +49 30 293697300   

                                         email  \
element id                                       
node    669088712                          NaN   
        694302689                          NaN   
        872223518                          NaN   
        874357625                          NaN   
        911925361                          NaN   
        933513566                          NaN   
        1038248666  info@chirurgie-ostkreuz.de   
        1168212013                         NaN   
        1217659471                         NaN   
        1272472319 

# Step 7: review created data frame

## 7.1 How many rows and columns?

In [495]:


print("Rows, Columns:", hospital_df.shape)

Rows, Columns: (251, 26)


## 7.2 missing values per columns

In [496]:
missing_count = hospital_df.isna().sum().sort_values(ascending=False)
print(missing_count)

beds                  248
brand                 246
source                240
email                 234
toilets_wheelchair    229
emergency             211
phone                 194
opening_hours         183
operator              166
speciality            162
website               150
wheelchair            128
city                   76
country                73
postcode               70
housenumber            62
street                 59
name                    6
amenity_tag             4
latitude                0
longitude               0
geometry                0
neighborhood            0
healthcare_tag          0
district                0
district_id             0
dtype: int64


In [497]:
# Number of rows (observations, hospitals)
# I need this to compute percentages of missing values below

row_count = len(hospital_df)
print(row_count)

251


In [498]:
# Build table with counts and % of missing values
# What does pd.DataFrame({...}) do? It converts that dictionary into a DataFrame (like an Excel table).
# The keys become column names.
# The values become column data.

missing = pd.DataFrame({
    "missing_count": missing_count,
    "missing_pct": (missing_count / row_count * 100).round(1)
}).sort_values(by="missing_pct", ascending=False)

print(missing)

                    missing_count  missing_pct
beds                          248         98.8
brand                         246         98.0
source                        240         95.6
email                         234         93.2
toilets_wheelchair            229         91.2
emergency                     211         84.1
phone                         194         77.3
opening_hours                 183         72.9
operator                      166         66.1
speciality                    162         64.5
website                       150         59.8
wheelchair                    128         51.0
city                           76         30.3
country                        73         29.1
postcode                       70         27.9
housenumber                    62         24.7
street                         59         23.5
name                            6          2.4
amenity_tag                     4          1.6
latitude                        0          0.0
longitude    

# 7.3. Decision for keeping columns no/yes
beds                          248         98.8.    --> drop
brand                         246         98.0.    --> drop
source                        240         95.6     --> hardcode "OSM

In [499]:
hospital_df = hospital_df.drop(columns=["beds", "brand"])
print(hospital_df)

                                                                                                                name  \
element  id                                                                                                            
node     669088712                                                                            Ärztehaus Johannisthal   
         694302689                                                                       Ärztehaus am Roedeliusplatz   
         872223518                                                                                         Ärztehaus   
         874357625                                                                                MVZ am Moritzplatz   
         911925361                                                                           Ärztehaus Damerowstraße   
         933513566                                                                        Ärztehaus am Schlachtensee   
         1038248666                     

In [500]:
hospital_df.loc[hospital_df["source"].isna(), "source"] = "OSM"

# 8. Handling of missing value / normalization


In [501]:
# Replace NaN with "unknown" and standardize values

text_cols = ["name", "street", "city", "country", "website", "operator", "brand", "phone", "email", "source", "beds","housenumber", "postcode","emergency","wheelchair", "speciality" ,'opening_hours']
for col in text_cols:
    if col in hospital_df.columns:
        hospital_df[col] = hospital_df[col].astype(str).str.strip()
        hospital_df[col] = hospital_df[col].replace({"nan": "unknown", "none": "unknown", "null": "unknown"})   
hospital_df.head(10)

name  \
element id                                                  
node    669088712                  Ärztehaus Johannisthal   
        694302689             Ärztehaus am Roedeliusplatz   
        872223518                               Ärztehaus   
        874357625                      MVZ am Moritzplatz   
        911925361                 Ärztehaus Damerowstraße   
        933513566              Ärztehaus am Schlachtensee   
        1038248666     Chirurgie & Orthopädie am Ostkreuz   
        1168212013                            MVZ-Vitalis   
        1217659471               Nierenzentrum Zehlendorf   
        1272472319  DTZ Berlin - Ärztehaus Kadiner Straße   

                                                                           operator  \
element id                                                                            
node    669088712                                                           unknown   
        694302689                                                           unknown   
        872223518                                                           unknown   
        874357625                                                           unknown   
        911925361                                                           unknown   
        933513566                                                           unknown   
        1038248666  Dr. med. Christoph Graf von Matuschka  Dr. med. Christoph Lemke   
        1168212013                                                          unknown   
        1217659471                                                          unknown   
        1272472319                                                          unknown   

                    country     city               street housenumber  \
element id                                                              
node    669088712        DE   Berlin            Sterndamm         152   
        694302689        DE   Berlin         Schottstraße           4   
        872223518   unknown  unknown              unknown     unknown   
        874357625        DE   Berlin        Oranienstraße         158   
        911925361        DE   Berlin        Damerowstraße           7   
        933513566        DE   Berlin    Breisgauer Straße         1-3   
        1038248666       DE   Berlin        Sonntagstraße           1   
        1168212013       DE   Berlin           Alt-Buckow        9-11   
        1217659471       DE   Berlin  Fischerhüttenstraße         111   
        1272472319       DE   Berlin       Kadiner Straße          23   

                   postcode    neighborhood             phone  \
element id                                                      
node    669088712     12487    Johannisthal           unknown   
        694302689     10365     Lichtenberg           unknown   
        872223518   unknown       Kreuzberg           unknown   
        874357625     10969       Kreuzberg    +49 30 6146017   
        911925361     13187          Pankow           unknown   
        933513566     14129   Schlachtensee           unknown   
        1038248666    10245  Friedrichshain           unknown   
        1168212013    12349          Buckow   +49 30 67065690   
        1217659471    14163      Zehlendorf      +49308139121   
        1272472319    10243  Friedrichshain  +49 30 293697300   

                                         email  \
element id                                       
node    669088712                      unknown   
        694302689                      unknown   
        872223518                      unknown   
        874357625                      unknown   
        911925361                      unknown   
        933513566                      unknown   
        1038248666  info@chirurgie-ostkreuz.de   
        1168212013                     unknown   
        1217659471                     unknown   
        1272472319                     unknown   

                

In [502]:
# Standardize column names

hospital_df.columns = hospital_df.columns.str.lower().str.strip().str.replace(" ", "_").str.replace("-", "_")

# Convert certain columns to correct type

hospital_df["housenumber"] = hospital_df["housenumber"].astype(str)   # ensure text

hospital_df["postcode"] = hospital_df["postcode"].astype(str)         # keep leading zeros

# Normalize yes/no columns into Boolean (True/False)

hospital_df["wheelchair"] = hospital_df["wheelchair"].map({"yes": True, "no": False})

# Make text values consistent (lowercase to avoid duplicates )
 

In [503]:
#rename final dataframe to hospital_osm
hospital_osm = hospital_df
hospital_osm  

name  \
element  id                                                                                                            
node     669088712                                                                            Ärztehaus Johannisthal   
         694302689                                                                       Ärztehaus am Roedeliusplatz   
         872223518                                                                                         Ärztehaus   
         874357625                                                                                MVZ am Moritzplatz   
         911925361                                                                           Ärztehaus Damerowstraße   
         933513566                                                                        Ärztehaus am Schlachtensee   
         1038248666                                                               Chirurgie & Orthopädie am Ostkreuz   
         1168212013                                                                                      MVZ-Vitalis   
         1217659471                                                                         Nierenzentrum Zehlendorf   
         1272472319                                                            DTZ Berlin - Ärztehaus Kadiner Straße   
         1336685638                                                                                        Ärztehaus   
         1618861746                                                                           Ärzte Zentrum Köpenick   
         1724523076                                                                     Tagesklinik unter den Eichen   
         1732901969                                                                              Medico Leopoldplatz   
         1734428743                                                                                        Ärztehaus   
         1974936511                                                   Praxis für Radioonkologie und Strahlentherapie   
         2021436781                                              Sana Gesundheitszentrum Marzahn "Ernst Ludwig Heim"   
         2105895889                                                                   Ärztehaus am Tempelhofer Hafen   
         2126141390                                                                                        Ärztehaus   
         2335086445                                   Sana Gesundheitszentrum Friedrichshain "Frédéric Joliot-Curie"   
         2435045640                                                                  Rückenzentrum am Markgrafenpark   
         2495554370                                                             MVZ Diagnostisches Zentrum Kreuzberg   
         2509419045                                                                         Ärztehaus Neubritzer Tor   
         2557712726                                                                                Magnet MVZ Berlin   
         2821025302                                                                     Nierenzentrum Charlottenburg   
         2851155103                                                          Zahnklinik/Zahnärztlicher Notfalldienst   
         2935287105                                                                               Gesundheitszentrum   
         3310721906                                                                          Ärztehaus Galenusstraße   
         3371705079                                                                              Ärztehaus Rahnsdorf   
         3374664253                                                               Frauenheilkunde und Geburtsmedizin   
         3374664287                                         Klinik für Anästhesiologie und operative Intensivmedizin   
         3374664288                                                                        Klinik für Augenheilkunde   
         3374664289             

In [504]:
hospital_osm.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 251 entries, ('node', np.int64(669088712)) to ('way', np.int64(1433767664))
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   name                251 non-null    object  
 1   operator            251 non-null    object  
 2   country             251 non-null    object  
 3   city                251 non-null    object  
 4   street              251 non-null    object  
 5   housenumber         251 non-null    object  
 6   postcode            251 non-null    object  
 7   neighborhood        251 non-null    object  
 8   phone               251 non-null    object  
 9   email               251 non-null    object  
 10  website             251 non-null    object  
 11  wheelchair          108 non-null    object  
 12  toilets_wheelchair  22 non-null     object  
 13  emergency           251 non-null    object  
 14  speciality          251 non-